In [1]:
!date

Mon May 19 04:54:08 PM EDT 2025


In [2]:
pwd

'/projects/sccn/andromeda1/aglinska/BC-ABCD-denoise/Code'

In [3]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import ants
import pickle
from tqdm import tqdm

In [4]:
def correlate_columns(arr1, arr2):
    """
    Computes the Pearson correlation between corresponding columns of two matrices.
    
    Parameters:
    arr1 (np.ndarray): First matrix of shape (370, 1000)
    arr2 (np.ndarray): Second matrix of shape (370, 1000)
    
    Returns:
    np.ndarray: 1D array of correlations for each column (size 1000)
    """
    # Ensure input arrays are numpy arrays
    arr1 = np.asarray(arr1)
    arr2 = np.asarray(arr2)
    
    # Subtract the mean of each column (normalize)
    arr1_centered = arr1 - np.mean(arr1, axis=0)
    arr2_centered = arr2 - np.mean(arr2, axis=0)
    
    # Compute the numerator (covariance)
    numerator = np.sum(arr1_centered * arr2_centered, axis=0)
    
    # Compute the denominator (product of standard deviations)
    denominator = np.sqrt(np.sum(arr1_centered**2, axis=0) * np.sum(arr2_centered**2, axis=0))
    
    # Compute the Pearson correlation for each column
    correlation = numerator / denominator
    
    return correlation

In [5]:
def get_regs(events_fn):
    from nilearn.glm.first_level import make_first_level_design_matrix

    events = pd.read_csv(events_fn,delimiter='\t')

    t_r = 2.0 
    #n_scans = epi.shape[-1]
    n_scans = 156
    frame_times = (np.arange(n_scans) * t_r)

    X1 = make_first_level_design_matrix(frame_times,events,drift_model="polynomial",drift_order=3,hrf_model="SPM") #

    face_reg = X1[['face']].values.sum(axis=1)
    place_reg = X1[['house','scene']].values.sum(axis=1)
    #place_reg = X1[['body', 'house', 'object', 'scene', 'scramble']].values.sum(axis=1)
    
    return face_reg,place_reg

events_fn_temp = '../Data/StudyForrest/events/{sub}_ses-localizer_task-objectcategories_run-{r}_events.tsv'
face_reg,place_reg = get_regs(events_fn_temp.format(sub='sub-01',r=1))

In [6]:
def get_contrast_val(s,r,Y,return_type='contrast',contype='face'):
    from nilearn.glm.first_level import make_first_level_design_matrix
    
    events = pd.read_csv(events_fn_temp.format(sub=subs[s],r=r),delimiter='\t')

    t_r = 2.0 
    n_scans = 156  # the acquisition comprises 128 scans
    frame_times = (np.arange(n_scans) * t_r)

    X1 = make_first_level_design_matrix(frame_times,events,drift_model="polynomial",drift_order=3,hrf_model="SPM") #
    
    Y = (Y-Y.mean(axis=1)[:,np.newaxis])/Y.std(axis=1)[:,np.newaxis] # Z score values
    Y = Y.transpose()
    X = X1.values
    beta = np.linalg.inv(X.T @ X1) @ X1.T @ Y
    beta = beta.T
    beta = beta.values

    #['body', 'face', 'house', 'object', 'scene', 'scramble', 'drift_1','drift_2', 'drift_3', 'constant']
    
    if contype=='face':
        contrast_vector = np.array([-1,5,-1,-1,-1,-1,0,0,0,0]) # Faces > Rest
        contrast_values = beta @ contrast_vector
    elif contype=='place':
        contrast_vector = np.array([-1,-1,3,-1,3,-1,0,0,0,0]) # Places over Rest
        contrast_values = beta @ contrast_vector
    
    if return_type=='contrast':
        return contrast_values.mean()
    elif return_type=='betas':
        return beta.mean(axis=0)

In [7]:
ffa_mask_temp = '../Data/StudyForrest/ROIs/rFFA_final_mask_{sub}_bin.nii.gz'
ppa_mask_temp = '../Data/StudyForrest/ROIs/rPPA_final_mask_{sub}_bin.nii.gz'

In [8]:
subs = ['sub-01','sub-02','sub-03','sub-04','sub-05','sub-09','sub-10','sub-14','sub-15','sub-16','sub-17','sub-18','sub-19','sub-20',]

In [9]:
faceMap = ants.image_read('../Misc/ffa_map_resampled_forrest.nii.gz')
faceMap_mask = faceMap.numpy()>5
faceMap_mask.sum()

953

In [10]:
roi = ants.image_read('../Data/StudyForrest/fmriprep/mask_roi.nii')
roni = ants.image_read('../Data/StudyForrest/fmriprep/mask_roni.nii')

In [11]:
def get_result(fn_temp,subFormat='name'):
    result = np.zeros((14,4,6))
    for s in tqdm(range(14)):
        for r in [1,2,3,4]:
            
            if subFormat=='number':
                subim = ants.image_read(os.path.join(fn_temp.format(s=s,r=r)))
            elif subFormat=='name':
                subim = ants.image_read(os.path.join(fn_temp.format(sub=subs[s],r=r)))
            
            subFFA = ants.image_read(os.path.join(ffa_mask_temp.format(sub=subs[s])))
            valsFFA = subim.numpy()[subFFA.numpy()==1]
            valsFFA = valsFFA[valsFFA.std(axis=1)>1e-3,:]
            
            subPPA = ants.image_read(os.path.join(ppa_mask_temp.format(sub=subs[s])))
            valsPPA = subim.numpy()[subPPA.numpy()==1]
            valsPPA = valsPPA[valsPPA.std(axis=1)>1e-3,:]

            corr2reg_face = np.nanmean(correlate_columns(valsFFA.transpose(),np.array([face_reg for _ in range(valsFFA.shape[0])]).transpose()))
            contrastVal_face = get_contrast_val(s,r,valsFFA,return_type='contrast',contype='face')
            
            corr2reg_place = np.nanmean(correlate_columns(valsPPA.transpose(),np.array([place_reg for _ in range(valsPPA.shape[0])]).transpose()))
            contrastVal_place = get_contrast_val(s,r,valsPPA,return_type='contrast',contype='place')
            
            arr1=subim.numpy()[(roi.numpy()==1) * (subFFA.numpy()==True)]
            arr2=subim.numpy()[(roi.numpy()==1) * (subFFA.numpy()==False)]
            arr1=arr1[arr1.std(axis=1)>1e-3]
            arr2=arr2[arr2.std(axis=1)>1e-3]
            r1=correlate_columns(arr1.transpose(),np.array([face_reg for _ in range(arr1.shape[0])]).transpose())
            r2=correlate_columns(arr2.transpose(),np.array([face_reg for _ in range(arr2.shape[0])]).transpose())
            specificity_face = r1.mean()-r2.mean()
            
            arr1=subim.numpy()[(roi.numpy()==1) * (subPPA.numpy()==True)]
            arr2=subim.numpy()[(roi.numpy()==1) * (subPPA.numpy()==False)]
            arr1=arr1[arr1.std(axis=1)>1e-3]
            arr2=arr2[arr2.std(axis=1)>1e-3]
            r1=correlate_columns(arr1.transpose(),np.array([place_reg for _ in range(arr1.shape[0])]).transpose())
            r2=correlate_columns(arr2.transpose(),np.array([place_reg for _ in range(arr2.shape[0])]).transpose())
            specificity_place = r1.mean()-r2.mean()

            result[s,r-1,0] = corr2reg_face
            result[s,r-1,1] = contrastVal_face
            result[s,r-1,2] = corr2reg_place
            result[s,r-1,3] = contrastVal_place
            result[s,r-1,4] = specificity_face
            result[s,r-1,5] = specificity_place
            
    return result

In [12]:
fn_temp = '../Data/StudyForrest/ensembles_last_CVAE/fixed-seed-00-YuOrig-00-DataloaderFix/signal_S{s}_R{r}_avg.nii.gz'
result_CVAE_Yu = get_result(fn_temp,subFormat='number')

100%|██████████| 14/14 [01:05<00:00,  4.70s/it]


In [13]:
fn_temp = '../Data/StudyForrest/fmriprep/{sub}/ses-localizer/func/{sub}_ses-localizer_task-objectcategories_run-{r}_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'
result_preproc = get_result(fn_temp,subFormat='name')

100%|██████████| 14/14 [01:22<00:00,  5.87s/it]


In [14]:
fn_temp = '../Data/StudyForrest/fmriprep/{sub}/ses-localizer/func/{sub}_ses-localizer_task-objectcategories_run-{r}_bold_space-MNI152NLin2009cAsym_COMPCOR-RONI.nii'
result_compcor = get_result(fn_temp,subFormat='name')

100%|██████████| 14/14 [01:11<00:00,  5.12s/it]


In [15]:
fn_temp = '../Data/StudyForrest/ensembles_last_CVAE/DeepCor-Forrest-face-v5/signal_S{s}_R{r}_avg.nii.gz'
result_CVAE_AA = get_result(fn_temp,subFormat='number')

 14%|█▍        | 2/14 [00:09<00:57,  4.78s/it]/home/aglinska/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:232: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/home/aglinska/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:243: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
<ipython-input-4-9f9a06f806b9>:24: RuntimeWarning: overflow encountered in square
  denominator = np.sqrt(np.sum(arr1_centered**2, axis=0) * np.sum(arr2_centered**2, axis=0))
/home/aglinska/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
100%|██████████| 14/14 [01:05<00:00,  4.71s/it]


In [16]:
model_names=['conv_denoise',
 'conv_denoise500',
 'conv_denoise_bn',
 'conv_denoise_large',
 'conv_denoise_large500',
 'conv_smooth',
 'conv_smooth-small',
 'conv_weights_unet_denoise',
 'conv_weights_unet_denoise500']

In [22]:
fn_temp = '../Data/StudyForrest/Stefano_adv_outputs/01-signals_averaged/S{s}-R{r}-conv_denoise-avg.nii.gz'
result_conv_denoise = get_result(fn_temp,subFormat='number')

100%|██████████| 14/14 [01:12<00:00,  5.18s/it]


In [23]:
fn_temp = '../Data/StudyForrest/Stefano_adv_outputs/01-signals_averaged/S{s}-R{r}-conv_denoise500-avg.nii.gz'
result_conv_denoise500 = get_result(fn_temp,subFormat='number')

100%|██████████| 14/14 [01:17<00:00,  5.56s/it]


In [24]:
fn_temp = '../Data/StudyForrest/Stefano_adv_outputs/01-signals_averaged/S{s}-R{r}-conv_denoise_bn-avg.nii.gz'
result_conv_denoise_bn = get_result(fn_temp,subFormat='number')

100%|██████████| 14/14 [01:21<00:00,  5.84s/it]


In [25]:
fn_temp = '../Data/StudyForrest/Stefano_adv_outputs/01-signals_averaged/S{s}-R{r}-conv_smooth-avg.nii.gz'
result_conv_smooth = get_result(fn_temp,subFormat='number')

100%|██████████| 14/14 [01:24<00:00,  6.02s/it]


In [28]:
fn_temp = '../Data/StudyForrest/Stefano_adv_outputs/01-signals_averaged/S{s}-R{r}-conv_denoise_large-avg.nii.gz'
result_conv_denoise_large = get_result(fn_temp,subFormat='number')

100%|██████████| 14/14 [01:28<00:00,  6.34s/it]


In [26]:
fn_temp = '../Data/StudyForrest/Stefano_adv_outputs/01-signals_averaged/S{s}-R{r}-conv_denoise_large500-avg.nii.gz'
result_conv_denoise_large500 = get_result(fn_temp,subFormat='number')

100%|██████████| 14/14 [01:13<00:00,  5.25s/it]


In [27]:
fn_temp = '../Data/StudyForrest/Stefano_adv_outputs/01-signals_averaged/S{s}-R{r}-conv_weights_unet_denoise-avg.nii.gz'
result_conv_weights_unet_denoise = get_result(fn_temp,subFormat='number')

100%|██████████| 14/14 [01:11<00:00,  5.07s/it]


In [19]:
names = ['result_preproc',
'result_compcor',
'result_conv_denoise',
'result_CVAE_Yu',
'result_CVAE_AA',]

In [ ]:
arrs = [result_preproc.mean(axis=0).mean(axis=0),
result_compcor.mean(axis=0).mean(axis=0),
result_conv_denoise.mean(axis=0).mean(axis=0),
result_CVAE_Yu.mean(axis=0).mean(axis=0),
result_CVAE_AA.mean(axis=0).mean(axis=0),]

In [21]:
df = pd.DataFrame(arrs,columns=['corr2reg_face','contrastVal_face','corr2reg_place','contrastVal_place','specificity_face','specificity_place'])
df['names'] = names
df

,corr2reg_face,contrastVal_face,corr2reg_place,contrastVal_place,specificity_face,specificity_place,names
0,0.147232,1.866531,0.249454,3.686941,0.151931,0.251526,result_preproc
1,0.167879,2.060754,0.271078,4.057826,0.166171,0.265403,result_compcor
2,0.199547,2.715568,0.307565,4.372802,0.190002,0.286192,result_conv_denoise
3,0.210093,1.982772,0.350785,5.266423,0.188370,0.335175,result_CVAE_Yu
4,0.245298,2.600636,0.409713,6.169140,0.237673,0.414718,result_CVAE_AA


In [32]:
arrs = [result_preproc.mean(axis=0).mean(axis=0),
result_compcor.mean(axis=0).mean(axis=0),
result_conv_denoise.mean(axis=0).mean(axis=0),
result_conv_denoise500.mean(axis=0).mean(axis=0),
result_conv_denoise_bn.mean(axis=0).mean(axis=0),
result_conv_smooth.mean(axis=0).mean(axis=0),
result_conv_denoise_large.mean(axis=0).mean(axis=0),
result_conv_weights_unet_denoise.mean(axis=0).mean(axis=0),
result_CVAE_Yu.mean(axis=0).mean(axis=0),
result_CVAE_AA.mean(axis=0).mean(axis=0),]

names = ['preproc',
'compcor',
'conv_denoise',
'conv_denoise_long',
'conv_denoise_BN',
'conv_smooth',
'conv_denoise_large',
'conv_weights_unet_denoise',
'CVAE_Yu',
'CVAE_AA']

df = pd.DataFrame(arrs,columns=['corr2reg_face','contrastVal_face','corr2reg_place','contrastVal_place','specificity_face','specificity_place'])
df['names'] = names
df

,corr2reg_face,contrastVal_face,corr2reg_place,contrastVal_place,specificity_face,specificity_place,names
0,0.147232,1.866531,0.249454,3.686941,0.151931,0.251526,preproc
1,0.167879,2.060754,0.271078,4.057826,0.166171,0.265403,compcor
2,0.199547,2.715568,0.307565,4.372802,0.190002,0.286192,conv_denoise
3,0.192869,2.523999,0.290921,4.210294,0.183736,0.270932,conv_denoise_long
4,0.178558,2.403890,0.233639,3.306160,0.162095,0.217818,conv_denoise_BN
5,0.167949,2.303065,0.245884,3.631025,0.159885,0.231558,conv_smooth
6,0.182833,2.576522,0.256966,3.798450,0.169548,0.232360,conv_denoise_large
7,0.180436,2.399011,0.253868,3.715063,0.166474,0.237937,conv_weights_unet_denoise
8,0.210093,1.982772,0.350785,5.266423,0.188370,0.335175,CVAE_Yu
9,0.245298,2.600636,0.409713,6.169140,0.237673,0.414718,CVAE_AA


In [31]:
# For GRANT
# For 10 subjects.
# bar plot preproc,compcor,CVAE_Yu,CVAE_AA,conv_denoise_long: separate bar plots  corr2reg_face  corr2reg_place specificity_face specificity_place
#
#
# 

In [ ]:
print(result_preproc.mean(axis=0).mean(axis=0)/result_preproc.mean(axis=1).std(axis=0))
print(result_compcor.mean(axis=0).mean(axis=0)/result_preproc.mean(axis=1).std(axis=0))
print(result_conv_denoise.mean(axis=0).mean(axis=0)/result_preproc.mean(axis=1).std(axis=0))
print(result_CVAE_Yu.mean(axis=0).mean(axis=0)/result_preproc.mean(axis=1).std(axis=0))
print(result_CVAE_AA.mean(axis=0).mean(axis=0)/result_preproc.mean(axis=1).std(axis=0))

In [ ]:
print(result_preproc.mean(axis=0).mean(axis=0))
print(result_compcor.mean(axis=0).mean(axis=0))
print(result_conv_denoise.mean(axis=0).mean(axis=0))
print(result_CVAE_Yu.mean(axis=0).mean(axis=0))
print(result_CVAE_AA.mean(axis=0).mean(axis=0))

In [ ]:
# result[s,r-1,0] = corr2reg_face
# result[s,r-1,1] = contrastVal_face
# result[s,r-1,2] = corr2reg_place
# result[s,r-1,3] = contrastVal_place
# result[s,r-1,4] = specificity